<center> 
    
# ⚔`timm` for Transfer Learning
    
<br>
<br>

</center>

<div class="alert alert-block alert-success">  
 <b>Fast.ai Version:</b> v2.0.17  <b>Draft Date:</b> 2020-12-04   
<br>
   
<br> 
<br>
    
记录`fast.ai` 使用中的一些尝试 ...
    
预训练模型一览：[pytorch-image-models](https://github.com/rwightman/pytorch-image-models)
</div>


In [1]:
from fastai.vision.all import *

In [2]:
import fastai
fastai.__version__

'2.0.17'

## 标记

<div class="alert alert-block alert-info">
<b>提示💡:</b> 

蓝色给大家一些新的提示要点，可以进行拓展操作.
    
<br>
<b>经验分享👣</b>

分享一些经验，踩过的坑. 
</div>

<div class="alert alert-block alert-warning">  
<b>知识回顾📕:</b> 

这部分会对学过的知识进行关联和回顾，便于大家进行巩固.  
</div>

<div class="alert alert-block alert-success">  
概要✅：

绿色部分为章节内容核心点，提醒关注.
</div>

<div class="alert alert-block alert-danger">  
<b>重要📌</b>
    
红色标识了重点注意事项，提醒大家注意.   
</div>

<details><summary>🤔--这是被隐藏的内容，思考之后再打开👇</summary>
<center>
>
>
打开之后你将看到内容的详情 ...
>
>
</center>
</details>

## 尝试

In [4]:
path = untar_data(URLs.PETS)
files = get_image_files(path/'images')

In [5]:
pat = r'/([^/]+)_\d+.*'
item_tfms = RandomResizedCrop(460, min_scale=0.75, ratio=(1.,1.))
batch_tfms = [*aug_transforms(size=224, max_warp=0), Normalize.from_stats(*imagenet_stats)]
bs=16
pets = DataBlock(blocks=(ImageBlock, CategoryBlock),
                 get_items=get_image_files,
                 splitter=RandomSplitter(0.2),
                 get_y=RegexLabeller(pat = pat),
                 item_tfms=item_tfms,
                 batch_tfms=batch_tfms)
dls = pets.dataloaders(path/'images', bs=bs)

### 创建模型BODY

fastai has a create_body function, whcih is called during cnn_learner, that will take a model architecuture and slice off the last Linear layer (resulting in a "body" that outputs unpooled features).

In [7]:
create_body??

Signature: create_body(arch, n_in=3, pretrained=True, cut=None)
Source:   
def create_body(arch, n_in=3, pretrained=True, cut=None):
    "Cut off the body of a typically pretrained `arch` as determined by `cut`"
    model = arch(pretrained=pretrained)
    _update_first_layer(model, n_in, pretrained)
    #cut = ifnone(cut, cnn_config(arch)['cut'])
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
    if   isinstance(cut, int):      return nn.Sequential(*list(model.children())[:cut])
    elif callable(cut): return cut(model)
    else:                           raise NamedError("cut must be either integer or a function")
File:      ~/gaoc/venv/fastai/lib/python3.7/site-packages/fastai/vision/learner.py
Type:      function


In [16]:
from timm import create_model
from fastai.vision.learner import _update_first_layer

def create_timm_body(arch:str, pretrained=True, cut=None, n_in=3):
    '''create a body from any model in the `timm` library. '''
    model = create_model(arch, pretrained=pretrained, num_classes=0, global_pool='')
    _update_first_layer(model, n_in, pretrained)
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
    if isinstance(cut, int): return nn.Sequential(*list(model.children())[:cut])
    elif callable(cut): return cut(model)
    else: raise NamedError("cut must be either integer or function")

In [17]:
body = create_timm_body('efficientnet_b3a', pretrained=True)

In [18]:
nf = num_features_model(body)*2; nf

3072

In [19]:
head = create_head(nf, dls.c)

In [20]:
net = nn.Sequential(body, head)

In [21]:
learn = Learner(dls, net, splitter=default_split)

In [22]:
learn.freeze()
unfrozen_params = filter(lambda p: not p.requires_grad, learn.model.parameters())
unfrozen_params = sum([np.prod(p.size()) for p in unfrozen_params])
model_parameters = filter(lambda p: p.requires_grad, learn.model.parameters())
frozen_params = sum([np.prod(p.size()) for p in model_parameters])

In [24]:
unfrozen_params, frozen_params

(10608936, 1686272)

In [27]:
learn.fine_tune(3)

epoch,train_loss,valid_loss,time
0,0.948835,0.784373,00:37


epoch,train_loss,valid_loss,time
0,0.632378,0.580456,00:45
1,0.482460,0.482865,00:46
2,0.381232,0.629675,00:46


### Refector

In [ ]:
def create_timm_model(arch:str, n_out, cut=None, pretrained=True, n_in=3, init=nn.init.kaiming_normal_, custom_head=None,
                     concat_pool=True, **kwargs):
    "Create custom architecture using `arch`, `n_in` and `n_out` from the `timm` library"
    body = create_timm_body(arch, pretrained, None, n_in)
    if custom_head is None:
        nf = num_features_model(nn.Sequential(*body.children())) * (2 if concat_pool else 1)
        head = create_head(nf, n_out, concat_pool=concat_pool, **kwargs)
    else: head = custom_head
    model = nn.Sequential(body, head)
    if init is not None: apply_init(model[1], init)
    return model

In [ ]:
def timm_learner(dls, arch:str, loss_func=None, pretrained=True, cut=None, splitter=None,
                y_range=None, config=None, n_out=None, normalize=True, **kwargs):
    "Build a convnet style learner from `dls` and `arch` using the `timm` library"
    if config is None: config = {}
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`"
    if y_range is None and 'y_range' in config: y_range = config.pop('y_range')
    model = create_timm_model(arch, n_out, default_split, pretrained, y_range=y_range, **config)
    learn = Learner(dls, model, loss_func=loss_func, splitter=default_split, **kwargs)
    if pretrained: learn.freeze()
    return learn

In [ ]:
learn = timm_learner(dls, 'efficientnet_b3a')

## 其它

### List timm Models

In [ ]:
import timm

In [ ]:
timm.list_models()[:10]

### Searching Models

In [ ]:
timm.list_models('*efficientnet*')[:10]

In [ ]:
timm.list_models('*b3a')[:10]

In [ ]:
timm.list_models('resne*t*', pretrained=True)[:10]

## Some Warnings

Watch for anything with a tf_ prefix. This means the original weights were ported from Google, so it uses manual padding to match TensorFlow's "same" padding, which adds GPU overhead and a general slowdown. If possible try to use the non-TF versions of models

HRNet is a bit of a problem-child, so it is the only one not straight-forward to use